<a href="https://colab.research.google.com/github/Vishy-A/DeepLearning/blob/main/HW2/4106HW2P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms



In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA (GPU) is available.")
else:
    device = torch.device("cpu")
    print("CUDA (GPU) is not available. Using CPU.")


CUDA (GPU) is available.


In [13]:
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                             (0.5, 0.5, 0.5))
    ]))

cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((.5, .5, .5),
                           (.5, .5, .5))
    ]))

trainloader = torch.utils.data.DataLoader(cifar10, batch_size=32, shuffle=True)
valloader = torch.utils.data.DataLoader(cifar10_val, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
class Block(nn.Module):
  expansion = 1
  def __init__(self, in_planes, planes, stride=1):
    super(Block, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)
    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion*planes:
      self.shortcut = nn.Sequential(
        nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
        nn.BatchNorm2d(self.expansion*planes)
      )

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out


In [15]:
class ResNet11(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet11, self).__init__()
    self.in_planes = 64
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.linear = nn.Linear(512*block.expansion, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out



In [16]:
class ResNet18(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet18, self).__init__()
    self.in_planes = 64
    self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    self.fc = nn.Linear(512, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = self.avg_pool(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    return out


In [17]:
model = ResNet11(Block, [1,1,1,1], num_classes=10)
print(model)

ResNet11(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-

In [18]:
net = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


In [19]:
trainingloss = []
valloss = []
valaccuracylist = []


In [20]:
for epoch in range(10):
  runningloss = 0.0
  net.train()
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data[0].to(device), data[1].to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    runningloss += loss.item()
  trainingloss.append(runningloss / len(trainloader))

  running_loss = 0.0
  net.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for data in valloader:
      images, labels = data[0].to(device), data[1].to(device)
      outputs = net(images)
      loss = criterion(outputs, labels)
      running_loss += loss.item()
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  valloss.append(running_loss / len(valloader))
  valaccuracy = 100 * correct / total
  valaccuracylist.append(valaccuracy)

  print(f'Epoch {epoch + 1}, Training loss: {trainingloss[-1]}, Validation loss: {valloss[-1]}, Validation Accuracy: {valaccuracy}%')


Epoch 1, Training loss: 1.3861164854874004, Validation loss: 1.1353527852140677, Validation Accuracy: 59.26%
Epoch 2, Training loss: 0.9435425621724938, Validation loss: 0.9263248864454202, Validation Accuracy: 67.63%
Epoch 3, Training loss: 0.7328787463564028, Validation loss: 0.7259418545439601, Validation Accuracy: 73.97%
Epoch 4, Training loss: 0.5751919002675583, Validation loss: 0.6814997155255023, Validation Accuracy: 76.79%
Epoch 5, Training loss: 0.4477947178515424, Validation loss: 0.7182557038701, Validation Accuracy: 75.63%
Epoch 6, Training loss: 0.3333739333393401, Validation loss: 0.7184085128977656, Validation Accuracy: 76.68%
Epoch 7, Training loss: 0.23183167125893875, Validation loss: 0.7544687592659515, Validation Accuracy: 76.31%
Epoch 8, Training loss: 0.16076792898913816, Validation loss: 0.7848652743588621, Validation Accuracy: 76.9%
Epoch 9, Training loss: 0.09718233247371587, Validation loss: 0.8780451665956753, Validation Accuracy: 76.54%
Epoch 10, Training l

In [21]:
print(f'Final Validation Accuracy: {valaccuracylist[-1]}%')
total_params = sum(p.numel() for p in net.parameters())
print(f'Total number of parameters in the model: {total_params}')


Final Validation Accuracy: 76.38%
Total number of parameters in the model: 4903242
